### Memo
- small data sample with many features 
- approach with traditional machine learning 

### Key Element 
- BaseEstimator, TransformerMixin, ClassifierMixin
- ElasticNetCV, LassoLarsCV
- 

In [1]:

import numpy as np
from sklearn.base import BaseEstimator,TransformerMixin, ClassifierMixin
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import ElasticNetCV, LassoLarsCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline, make_union
from sklearn.utils import check_array
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor

from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection

from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD

from sklearn.metrics import r2_score

X1~X8 is categorical data

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
id_test = test['ID']
train.pop('ID')
test.pop('ID')

for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))

In [3]:
n_comp = 12

# tSVD
tsvd = TruncatedSVD(n_components=n_comp, random_state=420)
tsvd_results_train = tsvd.fit_transform(train.drop(["y"], axis=1))
tsvd_results_test = tsvd.transform(test)

# PCA
pca = PCA(n_components=n_comp, random_state=420)
pca2_results_train = pca.fit_transform(train.drop(["y"], axis=1))
pca2_results_test = pca.transform(test)

# ICA
ica = FastICA(n_components=n_comp, random_state=420)
ica2_results_train = ica.fit_transform(train.drop(["y"], axis=1))
ica2_results_test = ica.transform(test)

# GRP
grp = GaussianRandomProjection(n_components=n_comp, eps=0.1, random_state=420)
grp_results_train = grp.fit_transform(train.drop(["y"], axis=1))
grp_results_test = grp.transform(test)

# SRP [Might cause -inf]
# srp = SparseRandomProjection(n_components=n_comp, dense_output=True, random_state=420)
# srp_results_train = srp.fit_transform(train.drop(["y"], axis=1))
# srp_results_test = srp.transform(test)

#save columns list before adding the decomposition components

usable_columns = list(set(train.columns) - set(['y']))

# Append decomposition components to datasets
for i in range(1, n_comp + 1):
    train['pca_' + str(i)] = pca2_results_train[:, i - 1]
    test['pca_' + str(i)] = pca2_results_test[:, i - 1]

    train['ica_' + str(i)] = ica2_results_train[:, i - 1]
    test['ica_' + str(i)] = ica2_results_test[:, i - 1]

    train['tsvd_' + str(i)] = tsvd_results_train[:, i - 1]
    test['tsvd_' + str(i)] = tsvd_results_test[:, i - 1]

    train['grp_' + str(i)] = grp_results_train[:, i - 1]
    test['grp_' + str(i)] = grp_results_test[:, i - 1]

#    train['srp_' + str(i)] = srp_results_train[:, i - 1]
#    test['srp_' + str(i)] = srp_results_test[:, i - 1]


/Users/kentchiu/anaconda/lib/python3.6/site-packages/sklearn/decomposition/fastica_.py:116: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '
/Users/kentchiu/anaconda/lib/python3.6/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [36]:

train_y = train['y'].values
usable_columns = list(set(train.columns) - set(['y']))


feature_cols =[]
for i in range(1, n_comp + 1):
    feature_cols+=['pca_' + str(i), 'ica_' + str(i), 'tsvd_' + str(i), 
                        'grp_' + str(i)]

#feature_cols = usable_columns 
# use only projected features
train_x = train[feature_cols].copy()
test   = test[feature_cols].copy()

# zero-center and normalized with train_x and dtest


A = train_x.mean()
B = 1/train_x.max()
train_x -= A
train_x *= B

test -= A 
test *= B


# normalized y (aX+b)
b = train_y.mean()
a = 1/train_y.max()
train_y -= b
train_y *= a

def recover_y(value, a, b):
    value*=(1/a)
    value+= b
    return value 



In [380]:

'''Train the xgb model then predict the test data'''

xgb_params = {'n_trees': 400, 
    'eta': 0.0045,
    'max_depth': 4,
    'subsample': 0.9,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1}
# NOTE: Make sure that the class is labeled 'class' in the data file

dtrain = xgb.DMatrix(train_x, train_y)
dtest = xgb.DMatrix(test)

num_boost_rounds = 1250

# train model
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)
y_pred = model.predict(dtest)
y_pred = list(recover_y(y_pred, a, b))



In [381]:
#### Value Replacing with ill-conditioned in test data
# https://www.kaggle.com/c/mercedes-benz-greener-manufacturing/discussion/35271
# https://crowdstats.eu/topics/kaggle-mercedes-benz-greener-manufacturing-leaderboard-probing
memo = { 1 : 71.34112, 12 : 109.30903, 23 : 115.21953, 28 : 92.00675, 42 : 87.73572, 43 : 129.79876, 
        45 : 99.55671, 57 : 116.02167, 3977 : 132.08556}
for i in range(len(id_test)):
    if id_test[i] in memo.keys():
        y_pred[i] = memo[id_test[i]]
        print (i, id_test[i], memo[id_test[i]])

0 1 71.34112
8 12 109.30903
17 23 115.21953
19 28 92.00675
24 42 87.73572
25 43 129.79876
26 45 99.55671
32 57 116.02167
1985 3977 132.08556


In [382]:
sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = y_pred

# sub['y'] = y_pred*0.75 + results*0.25
sub.to_csv('model_15.csv', index=False)
# stacked-models_04.csv = with outlier
# stacked-models_05.csv = without outlier second StackingEstimator max_depth=5
# stacked-models_06.csv = second StackingEstimator max_depth=3 
# stacked-models_07.csv = sub['y'] = y_pred*0.75 + results*0.25
# stacked-models_08.csv = n_component from 12 -> 20, second StackingEstimator max_depth=4
# stacked-models_09.csv = sub['y'] = y_pred*0.7145 + results*0.2855
'''
model_11.csv 
xgb_params = {
    'n_trees': 320, 
    'eta': 0.0045,
    'max_depth': 5,
    'subsample': 0.9,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1}
    
model_12.csv 
sub feature cols to unsample_col 

model_13.csv
use feature cols
xgb_params = {'n_trees': 400, 
    'eta': 0.0045,
    'max_depth': 4,
    'subsample': 0.9,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1}
    
LBacc = 0.53

model_14.csv
xgb_params = {'n_trees': 400, 
    'eta': 0.0045,
    'max_depth': 5,
    'subsample': 0.9,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1}

model_15.csv
replce prediction outlier, ill-condition
xgb_params = {'n_trees': 400, 
    'eta': 0.0045,
    'max_depth': 4,
    'subsample': 0.9,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1}
    
'''

"\nmodel_11.csv \nxgb_params = {\n    'n_trees': 320, \n    'eta': 0.0045,\n    'max_depth': 5,\n    'subsample': 0.9,\n    'objective': 'reg:linear',\n    'eval_metric': 'rmse',\n    'base_score': y_mean, # base prediction = mean(target)\n    'silent': 1}\n    \nmodel_12.csv \nsub feature cols to unsample_col \n\nmodel_13.csv\nuse feature cols\nxgb_params = {'n_trees': 400, \n    'eta': 0.0045,\n    'max_depth': 4,\n    'subsample': 0.9,\n    'objective': 'reg:linear',\n    'eval_metric': 'rmse',\n    'base_score': y_mean, # base prediction = mean(target)\n    'silent': 1}\n    \nLBacc = 0.53\n\nmodel_14.csv\nxgb_params = {'n_trees': 400, \n    'eta': 0.0045,\n    'max_depth': 5,\n    'subsample': 0.9,\n    'objective': 'reg:linear',\n    'eval_metric': 'rmse',\n    'base_score': y_mean, # base prediction = mean(target)\n    'silent': 1}\n\nmodel_15.csv\nreplce prediction outlier, ill-condition\nxgb_params = {'n_trees': 400, \n    'eta': 0.0045,\n    'max_depth': 4,\n    'subsample': 

In [4]:
from train_dp import AgentRegressor

In [7]:
train_y = train['y'].values
print (train_y)
usable_columns = list(set(train.columns) - set(['y']))

feature_cols =[]
for i in range(1, n_comp + 1):
    feature_cols+=['pca_' + str(i), 'ica_' + str(i), 'tsvd_' + str(i), 
                        'grp_' + str(i)]

train_x = train[feature_cols] 
test   = test[feature_cols]

agent = AgentRegressor(lr=1e-4, batch_size=2, train_x=train_x, train_y= train_y, test = test)


[ 130.81   88.53   76.26 ...,  109.22   87.48  110.85]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 14)                686       
_________________________________________________________________
dense_2 (Dense)              (None, 14)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 15        
Total params: 911.0
Trainable params: 911
Non-trainable params: 0.0
_________________________________________________________________


In [8]:
# add init  # add constrain 
# not use momentum https://stackoverflow.com/questions/37232782/nan-loss-when-training-regression-network
# still meas nan => gradient explod 
# fix net  => no use
# avoid neg-value in train => no use 
# Truncnla init => with lower std
# simplize the model
# in-regression problem, it is easy to explode the gradient ...
# use adam optimizer 
# before final layer => use softmax => give up this way
# give up normalize output => no use
# extremely low lr with patient 

## Keras Way 
# agent.train(lr=1e-8, callbacks=[prediction_history])
agent.tf_train(lr= 1e-5)

[ Start Training ]
EP2, Loss 0.37769603729248047, Pre[-162.69172668]
EP4, Loss 1.3507000207901, Pre[-453.74703979]
EP6, Loss 1.9622442722320557, Pre[-1004.29510498]
EP8, Loss 0.37150293588638306, Pre[-57.40127563]
EP10, Loss 0.31627941131591797, Pre[ 78.92613983]
EP12, Loss 0.8201830387115479, Pre[-798.35772705]
EP14, Loss 0.4687601327896118, Pre[-594.50048828]
EP16, Loss 0.6259749531745911, Pre[-239.01609802]
EP18, Loss 0.26182377338409424, Pre[-95.71070099]
EP20, Loss 0.5729621052742004, Pre[-521.58166504]
EP22, Loss 0.2325829267501831, Pre[-129.16893005]
EP24, Loss 0.09448276460170746, Pre[ 24.91943741]
EP26, Loss 0.494930237531662, Pre[-391.3651123]
EP28, Loss 0.03396304324269295, Pre[ 96.61119843]
EP30, Loss 0.011670475825667381, Pre[ 475.52728271]
EP32, Loss 0.029524128884077072, Pre[ 236.95574951]
EP34, Loss 0.05254710465669632, Pre[ 93.55278778]
EP36, Loss 0.0006662247469648719, Pre[ 307.94543457]
EP38, Loss 0.04014883190393448, Pre[ 248.07589722]
EP40, Loss 0.00931625720113515

In [446]:
res  = agent.predict(weight_file="model.h5")

(4209, 100)


In [328]:
print (res[0])
print (train_x.shape, test.shape)

nan
(4209, 100) (4209, 100)


In [40]:
for i,j in enumerate(train_x.min()): 
    print (i,'---',j) 
train_x.keys()[54]

0 --- -49.3613222056
1 --- -0.0409340348335
2 --- 16.8327786484
3 --- -62.8340671234
4 --- -3.81353373818
5 --- -49.4681592495
6 --- -0.044703914039
7 --- -55.1120813109
8 --- -103.894976882
9 --- -2.54235582545
10 --- -40.994456916
11 --- -0.0848390074851
12 --- -62.884367567
13 --- -45.4058265382
14 --- -1.27117791273
15 --- -36.5088849478
16 --- -0.0351387185724
17 --- -48.8139393595
18 --- 13.6961414957
19 --- -2.54235582545
20 --- -38.7889456372
21 --- -0.0419523224926
22 --- -44.3344801074
23 --- -50.6554929971
24 --- -5.08471165091
25 --- -13.1115024242
26 --- -0.0515765149057
27 --- -31.2001831175
28 --- -54.6094186533
29 --- -3.81353373818
30 --- -10.9016680919
31 --- -0.0543096932233
32 --- -14.9567393796
33 --- -72.4228065883
34 --- -2.54235582545
35 --- -7.01791967459
36 --- -0.0273556621901
37 --- -10.9103065955
38 --- -12.5252252538
39 --- -5.08471165091
40 --- -7.7268164085
41 --- -0.043745566988
42 --- -7.02846763314
43 --- -39.3253748109
44 --- -8.89824538909
45 --- -5

'srp_11'

In [ ]:
pred  = agent.predict()
#### Value Replacing with ill-conditioned in test data
# https://www.kaggle.com/c/mercedes-benz-greener-manufacturing/discussion/35271
# https://crowdstats.eu/topics/kaggle-mercedes-benz-greener-manufacturing-leaderboard-probing
memo = { 1 : 71.34112, 12 : 109.30903, 23 : 115.21953, 28 : 92.00675, 42 : 87.73572, 43 : 129.79876, 
        45 : 99.55671, 57 : 116.02167, 3977 : 132.08556}
for i in range(len(id_test)):
    if id_test[i] in memo.keys():
        y_pred[i] = memo[id_test[i]]
sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = y_pred

# sub['y'] = y_pred*0.75 + results*0.25
sub.to_csv('model_15.csv', index=False)